In [1]:
import os, cv2
import splitfolders
import shutil

In [2]:
# Define your input and output folder paths
input_folder = '/home/sowmya/AgSolaire/data'
output_folder = '/home/sowmya/AgSolaire/train_val_test_split'

# Check if the output folder exists and delete it if it does
try:
    shutil.rmtree(output_folder)
    print(f"Removed existing folder: {output_folder}")
except FileNotFoundError:
    print(f"Output folder not found, creating a new one: {output_folder}")

# Perform the train/validation/test split
splitfolders.ratio(
    input_folder, 
    output=output_folder, 
    seed=1337, 
    ratio=(0.8, 0.1, 0.1), 
    move=False
)

Removed existing folder: /home/sowmya/AgSolaire/train_val_test_split


In [8]:
from ultralytics import YOLO
import torch
import gc
# torch.cuda.memory_summary(device=None, abbreviated=False)
# gc.collect()
torch.cuda.empty_cache()
print(torch.__version__)

model   = YOLO('yolo11s.pt')

results = model.train(
    data   = '/home/sowmya/AgSolaire/yolo_training.yaml',
    imgsz  = 480,
    epochs = 20,
    batch  = 5,
    name   = 'agsolaire_yolo_training_new',
    device = 0
   
)

model.val()

2.8.0+cu128
New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.199 🚀 Python-3.13.7 torch-2.8.0+cu128 CUDA:0 (NVIDIA T1200 Laptop GPU, 3904MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=5, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/sowmya/AgSolaire/yolo_training.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ag

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x74fbd815e2e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [4]:
# from ultralytics import YOLO

# Load the best weights from your training
# model = YOLO("/home/sowmya/AgSolaire/notebooks/yolo11s.pt")
# Replace with your test image path
image_path = "/home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg"

# Run prediction
# results = model.predict(source=image_path, conf=0.5, show=True)  # conf is confidence threshold
results = model(image_path)
# Get bounding boxes for the first image in batch
boxes = results[0].boxes

# Count seeds
num_seeds = len(boxes)
print(f"Number of seeds detected: {num_seeds}")
import cv2
cv2.namedWindow("YOLO ",cv2.WINDOW_NORMAL)
# Read the image
image = cv2.imread(image_path)

# Draw boxes
for box in boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 5)

cv2.putText(image, f"Seeds: {num_seeds}", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Save the result
cv2.imwrite("seed_detection_result.jpg", image)
cv2.imshow("YOLO ", image)
cv2.resizeWindow("YOLO ", 800,600)

cv2.waitKey(0)
cv2.destroyAllWindows()



image 1/1 /home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg: 480x384 34 seeds, 35.3ms
Speed: 1.7ms preprocess, 35.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 384)
Number of seeds detected: 34


In [5]:
# Path to image
image_path = "/home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg"

# Run YOLO prediction
results = model(image_path)

# Get bounding boxes for the first image
boxes = results[0].boxes

# Count seeds
num_seeds = len(boxes)
print(f"Number of seeds detected: {num_seeds}")

# Read the image
image = cv2.imread(image_path)

# Draw seed centers
for box in boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)   # center of box
    cv2.circle(image, (cx, cy), 8, (0, 255, 0), -1)   # filled green circle

# Overlay total count
cv2.putText(image, f"Seeds: {num_seeds}", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Save & show
cv2.imwrite("seed_detection_result.jpg", image)

cv2.namedWindow("YOLO", cv2.WINDOW_NORMAL)
cv2.imshow("YOLO", image)
cv2.resizeWindow("YOLO", 800, 600)
cv2.waitKey(0)
cv2.destroyAllWindows()



image 1/1 /home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg: 480x384 34 seeds, 7.8ms
Speed: 1.4ms preprocess, 7.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 384)
Number of seeds detected: 34


In [10]:

import numpy as np
import csv
import os

# -----------------------------
# CONFIG
# -----------------------------
IMAGE_PATH = "/home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg"
WEIGHTS = "/home/sowmya/AgSolaire/notebooks/runs/detect/agsolaire_yolo_training_new/weights/best.pt"          # you used this for training; replace with your best checkpoint if needed
PX_PER_MM = None                # e.g., 12.5  (set if you know pixels-per-mm); otherwise stays in pixels only
ARUCO_MM = 10.0                 # if you keep an ArUco marker in-frame, specify its side length (mm) to auto-estimate px/mm
SAVE_OVERLAY = "seed_heights_overlay.jpg"
SAVE_CSV = "seed_heights.csv"

# -----------------------------
# (Optional) Estimate px/mm from an ArUco marker placed in the image
# -----------------------------
def estimate_px_per_mm_with_aruco(bgr, marker_length_mm=10.0, dict_id=cv2.aruco.DICT_5X5_100):
    try:
        aruco_dict = cv2.aruco.getPredefinedDictionary(dict_id)
        params = cv2.aruco.DetectorParameters()
        detector = cv2.aruco.ArucoDetector(aruco_dict, params)
        corners, ids, _ = detector.detectMarkers(bgr)
        if corners and len(corners) > 0:
            c = corners[0].reshape(-1, 2)  # 4 x 2
            side_px = np.mean([
                np.linalg.norm(c[0] - c[1]),
                np.linalg.norm(c[1] - c[2]),
                np.linalg.norm(c[2] - c[3]),
                np.linalg.norm(c[3] - c[0]),
            ])
            if side_px > 0 and marker_length_mm > 0:
                return side_px / marker_length_mm
    except Exception:
        pass
    return None

# -----------------------------
# Quick mask inside a detection box (classical threshold)
# Works best on clean backgrounds and consistent lighting
# -----------------------------
def mask_from_box(bgr, box_xyxy):
    x1, y1, x2, y2 = map(int, box_xyxy)
    # Guard
    h, w = bgr.shape[:2]
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)
    if x2 <= x1 or y2 <= y1:
        return None

    crop = bgr[y1:y2, x1:x2]
    lab = cv2.cvtColor(crop, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)

    # Otsu threshold on L channel
    _, th = cv2.threshold(L, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Ensure seed is white; flip if foreground darker than background
    if np.mean(L[th == 255]) < np.mean(L[th == 0]):
        th = 255 - th

    # Clean up
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=1)
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8), iterations=1)

    # Paste back to full-size mask
    mask = np.zeros(bgr.shape[:2], np.uint8)
    mask[y1:y2, x1:x2] = th
    return mask

# -----------------------------
# Measure the long axis (height) via rotated rectangle on the largest contour
# -----------------------------
def measure_long_axis_px(mask):
    if mask is None:
        return None, None, None
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None, None, None

    cnt = max(cnts, key=cv2.contourArea)
    rect = cv2.minAreaRect(cnt)        # ((cx, cy), (w, h), angle)
    (cx, cy), (w, h), angle = rect
    length_px = float(max(w, h))       # "height" = longer side

    box_pts = cv2.boxPoints(rect).astype(np.int32)  # 4 x 2
    center = (int(round(cx)), int(round(cy)))
    return length_px, center, box_pts

# -----------------------------
# MAIN
# -----------------------------
def main():
    # Load model & predict
    model = YOLO(WEIGHTS)
    results = model(IMAGE_PATH)         # inference
    boxes = results[0].boxes            # YOLO detections

    # Read image
    bgr = cv2.imread(IMAGE_PATH)
    overlay = bgr.copy()

    # If px/mm not provided, try estimating via ArUco (if marker is present)
    global PX_PER_MM
    if PX_PER_MM is None:
        est = estimate_px_per_mm_with_aruco(bgr, marker_length_mm=ARUCO_MM)
        if est:
            PX_PER_MM = est
            print(f"[INFO] Estimated px/mm via ArUco: {PX_PER_MM:.3f}")

    rows = []
    num_seeds = len(boxes)

    for i, box in enumerate(boxes, start=1):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # for numbering

        # Build a per-box mask and measure long axis
        mask = mask_from_box(bgr, (x1, y1, x2, y2))
        length_px, center, rect_pts = measure_long_axis_px(mask)

        # Fallback: if masking failed, use the bigger box side
        if length_px is None:
            length_px = float(max(x2 - x1, y2 - y1))
            center = (cx, cy)
            rect_pts = np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]], dtype=np.int32)

        # Convert to mm if available
        length_mm = (length_px / PX_PER_MM) if PX_PER_MM else None

        # Draw center dot, rotated rectangle, and label (index + height)
        cv2.circle(overlay, center, 5, (0, 255, 0), -1)
        cv2.polylines(overlay, [rect_pts], True, (255, 0, 0), 2)

        label = f"{i}: {length_px:.1f}px"
        if length_mm is not None:
            label += f" ({length_mm:.2f} mm)"

        cv2.putText(overlay, label, (center[0] + 10, center[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        rows.append({
            "seed_id": i,
            "cx_px": center[0],
            "cy_px": center[1],
            "height_px": round(length_px, 3),
            "height_mm": round(length_mm, 3) if length_mm is not None else None,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2
        })

    # Overlay total count
    cv2.putText(overlay, f"Seeds: {num_seeds}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Save outputs
    cv2.imwrite(SAVE_OVERLAY, overlay)
    print(f"[INFO] Saved annotated image → {SAVE_OVERLAY}")

    if rows:
        with open(SAVE_CSV, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
            writer.writeheader()
            for r in rows:
                writer.writerow(r)
        print(f"[INFO] Saved measurements → {SAVE_CSV}")
    else:
        print("[WARN] No detections to measure.")

    # (Optional) show
    cv2.namedWindow("Seed Heights", cv2.WINDOW_NORMAL)
    cv2.imshow("Seed Heights", overlay)
    cv2.resizeWindow("Seed Heights", 900, 700)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



image 1/1 /home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg: 480x384 34 seeds, 8.3ms
Speed: 1.2ms preprocess, 8.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 384)
[INFO] Saved annotated image → seed_heights_overlay.jpg
[INFO] Saved measurements → seed_heights.csv


In [ ]:
# measure_seed_dims.py
from ultralytics import YOLO
import cv2
import numpy as np
import csv
import os

# ======================
# CONFIG
# ======================
IMAGE_PATH = "/home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg"

WEIGHTS = "/home/sowmya/AgSolaire/notebooks/runs/detect/agsolaire_yolo_training_new/weights/best.pt" 
PX_PER_MM  = None           # e.g., 12.5  (pixels per millimeter). If None, we try ArUco.
ARUCO_MM   = 10.0           # side length (mm) of an ArUco marker in the frame (if used)
USE_GRABCUT = False         # set True for more robust masking inside boxes

OUT_OVERLAY = "seed_dims_overlay.jpg"
OUT_CSV     = "seed_dims.csv"

# ======================
# (Optional) px-per-mm from ArUco
# ======================
def estimate_px_per_mm_with_aruco(bgr, marker_length_mm=10.0, dict_id=cv2.aruco.DICT_5X5_100):
    """
    Estimate px/mm from a detected ArUco marker of known side length.
    Works best with the marker lying flat and roughly parallel to the image plane.
    """
    try:
        aruco_dict = cv2.aruco.getPredefinedDictionary(dict_id)
        params = cv2.aruco.DetectorParameters()
        detector = cv2.aruco.ArucoDetector(aruco_dict, params)
        corners, ids, _ = detector.detectMarkers(bgr)
        if corners and len(corners) > 0 and marker_length_mm > 0:
            c = corners[0].reshape(-1, 2)  # 4x2
            side_px = np.mean([
                np.linalg.norm(c[0] - c[1]),
                np.linalg.norm(c[1] - c[2]),
                np.linalg.norm(c[2] - c[3]),
                np.linalg.norm(c[3] - c[0]),
            ])
            if side_px > 0:
                return side_px / marker_length_mm
    except Exception:
        pass
    return None

# ======================
# Box -> mask
# ======================
def mask_from_box_threshold(bgr, box_xyxy):
    """Fast classical masking inside the box using LAB + Otsu."""
    x1, y1, x2, y2 = map(int, box_xyxy)
    h, w = bgr.shape[:2]
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)
    if x2 <= x1 or y2 <= y1:
        return None

    crop = bgr[y1:y2, x1:x2]
    lab = cv2.cvtColor(crop, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)

    _, th = cv2.threshold(L, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Ensure seed is foreground (white). Flip if needed.
    if np.mean(L[th == 255]) < np.mean(L[th == 0]):
        th = 255 - th

    # Clean small noise & fill small holes
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=1)
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8), iterations=1)

    mask = np.zeros(bgr.shape[:2], np.uint8)
    mask[y1:y2, x1:x2] = th
    return mask

def mask_from_box_grabcut(bgr, box_xyxy, iters=3):
    """More robust masking inside the box using GrabCut."""
    x1, y1, x2, y2 = map(int, box_xyxy)
    h, w = bgr.shape[:2]
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)
    if x2 <= x1 or y2 <= y1:
        return None

    mask = np.zeros((h, w), np.uint8)
    rect = (x1, y1, x2 - x1, y2 - y1)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    cv2.grabCut(bgr, mask, rect, bgdModel, fgdModel, iters, cv2.GC_INIT_WITH_RECT)
    fg = np.where((mask == cv2.GC_FGD) | (mask == cv2.GC_PR_FGD), 255, 0).astype('uint8')
    fg = cv2.morphologyEx(fg, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=1)
    return fg

def mask_from_box(bgr, box_xyxy):
    return mask_from_box_grabcut(bgr, box_xyxy) if USE_GRABCUT else mask_from_box_threshold(bgr, box_xyxy)

# ======================
# Measure height & width from mask
# ======================
def measure_axes_px(mask):
    """
    Returns (height_px, width_px, center, rect_pts)
    - height_px = longer side of rotated minAreaRect
    - width_px  = shorter side
    """
    if mask is None:
        return None, None, None, None

    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None, None, None, None

    cnt = max(cnts, key=cv2.contourArea)
    rect = cv2.minAreaRect(cnt)         # ((cx,cy),(w,h),angle)
    (cx, cy), (w, h), angle = rect
    height_px = float(max(w, h))
    width_px  = float(min(w, h))
    box_pts   = cv2.boxPoints(rect).astype(np.int32)
    center    = (int(round(cx)), int(round(cy)))
    return height_px, width_px, center, box_pts

# ======================
# Main
# ======================
def main():
    # Load model and run detection
    model = YOLO(WEIGHTS)
    results = model(IMAGE_PATH)
    boxes = results[0].boxes

    # Read image
    bgr = cv2.imread(IMAGE_PATH)
    if bgr is None:
        raise FileNotFoundError(f"Could not read IMAGE_PATH: {IMAGE_PATH}")
    overlay = bgr.copy()

    # Pixels-per-mm
    global PX_PER_MM
    if PX_PER_MM is None:
        est = estimate_px_per_mm_with_aruco(bgr, marker_length_mm=ARUCO_MM)
        if est:
            PX_PER_MM = est
            print(f"[INFO] Estimated px/mm via ArUco: {PX_PER_MM:.3f}")
        else:
            print("[WARN] PX_PER_MM not set and no ArUco found. Values will be in pixels.")

    rows = []
    num_seeds = len(boxes)

    for i, box in enumerate(boxes, start=1):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Build mask inside this detection box
        mask = mask_from_box(bgr, (x1, y1, x2, y2))

        # Measure axes
        height_px, width_px, center, rect_pts = measure_axes_px(mask)

        # Fallback if mask failed
        if height_px is None:
            height_px = float(max(x2 - x1, y2 - y1))
            width_px  = float(min(x2 - x1, y2 - y1))
            center    = ((x1 + x2) // 2, (y1 + y2) // 2)
            rect_pts  = np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]], np.int32)

        # Convert to mm if available
        if PX_PER_MM:
            height_mm = height_px / PX_PER_MM
            width_mm  = width_px  / PX_PER_MM
        else:
            height_mm = None
            width_mm  = None

        # Draw center dot, rotated rectangle
        cv2.circle(overlay, center, 5, (0, 255, 0), -1)
        cv2.polylines(overlay, [rect_pts], True, (255, 0, 0), 2)

        # Seed number + dimensions (mm if available, else px)
        if PX_PER_MM:
            label = f"{i}: {height_mm:.2f}x{width_mm:.2f} mm"
        else:
            label = f"{i}: {height_px:.1f}x{width_px:.1f} px"

        cv2.putText(overlay, label, (center[0] + 10, center[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        rows.append({
            "seed_id": i,
            "cx_px": center[0],
            "cy_px": center[1],
            "height_px": round(height_px, 3),
            "width_px": round(width_px, 3),
            "height_mm": round(height_mm, 3) if height_mm is not None else None,
            "width_mm": round(width_mm, 3) if width_mm is not None else None,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2
        })

    # Overlay total count
    cv2.putText(overlay, f"Seeds: {num_seeds}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Save outputs
    cv2.imwrite(OUT_OVERLAY, overlay)
    print(f"[INFO] Saved annotated image → {OUT_OVERLAY}")

    if rows:
        with open(OUT_CSV, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
            writer.writeheader()
            writer.writerows(rows)
        print(f"[INFO] Saved measurements → {OUT_CSV}")
    else:
        print("[WARN] No detections to measure.")

    # Optional: display
    try:
        cv2.namedWindow("Seed Dimensions", cv2.WINDOW_NORMAL)
        cv2.imshow("Seed Dimensions", overlay)
        cv2.resizeWindow("Seed Dimensions", 900, 700)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    except Exception:
        # In headless environments just skip
        pass

if __name__ == "__main__":
    main()



image 1/1 /home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg: 480x384 34 seeds, 8.3ms
Speed: 1.2ms preprocess, 8.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 384)
[WARN] PX_PER_MM not set and no ArUco found. Values will be in pixels.
[INFO] Saved annotated image → seed_dims_overlay.jpg
[INFO] Saved measurements → seed_dims.csv
